# Solving System of Linear Equations using Gaussian Elimination

## Gaussian Elimination Algorithm
Algorithm that is discussed below will only work on non-singular system of equations.

### Major steps in the Algorithm include:
1. [ Creating Augmented Matrix](#4)
2. [ Transforming Matrix into Row Echelon Form](#5)
3. [ Back Substitution](#6)
4. [ Compile the Gaussian Elimination Algorithm](#7)

### Functions used are:
1. [ Funtion swap_rows](#1)
2. [ Function for finding the first non-zero value's index in a column starting from a specified row](#2)
3. [ Function for finding the first non-zero value's index in a row](#3)
4. [ Function for construting Augmented Matrix](#4)

In [29]:
import numpy as np

<a name='1'></a>

## 1 - Function swap_rows

This function swaps the rows of the Matrix and returns the new matrix with swapped rows

In [30]:
def swap_rows(M,row_idx1,row_idx2):

  M = M.copy()
  M[[row_idx1,row_idx2]] = M[[row_idx2,row_idx1]]
  return M

In [42]:
M = np.array([
[1, 3, 6],
[0, -5, 2],
[-4, 5, 8]
])
print(swap_rows(M,0,2))

[[-4  5  8]
 [ 0 -5  2]
 [ 1  3  6]]


<a name='2'></a>
## 2 - Function for finding the first non-zero value's index in a column starting from a specified row

This function is used when pivot value has 0. Used to determine the non-zero value in the column and allowing for potential row swapping

In [43]:
def get_index_of_first_non_zero_element_from_column(M,column,start_row):

  column_array = M[start_row:,column]

  for i,val in enumerate(column_array):
    if not np.isclose(val,0,atol=1e-5):
      index = i+start_row
      return index

  return -1

In [48]:
N = np.array([
[0, 5, -3 ,6 ,8],
[0, 6, 3, 8, 1],
[0, 0, 0, 0, 0],
[0, 0, 0 ,0 ,7],
[0, 2, 1, 0, 4]
]
)
print(get_index_of_first_non_zero_element_from_column(N,2,2))

4


<a name='3'></a>

## Function for finding the first non-zero value's index in a row

This function helps in finding the index of first non-zero element in desired row (pivot element)

In [49]:
def get_index_of_first_non_zero_element_from_row(M,row,augmented=False):

  M = M.copy()

  if augmented == True:
    M = M[:,:-1]

  row_array = M[row]

  for i,val in enumerate(row_array):
    if not np.isclose(val,0,atol=1e-5):
      return i

  return -1


In [50]:
print(N)

[[ 0  5 -3  6  8]
 [ 0  6  3  8  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  7]
 [ 0  2  1  0  4]]


In [54]:
print(get_index_of_first_non_zero_element_from_row(N,3))

4


<a name='4'></a>

## Function for Constructing Augmented Matrix

This function creates Augmented matrix by horizontally stacking Coefficient matrix and output vector

In [55]:
def augmented_matrix(A,B):
  return np.hstack((A,B))

In [56]:
A = np.array([[1,2,3], [3,4,5], [4,5,6]])
B = np.array([[1], [5], [7]])

print(augmented_matrix(A,B))

[[1 2 3 1]
 [3 4 5 5]
 [4 5 6 7]]


<a name='5'></a>
## Function to Convert Matrix into Row Echelon Form

- Find Pivot element at each row (diagonal element)
  if it is equal to or close to 0,
    find the index of next non zero value from that column.
    Swap current pivot row with the index row returned
    and make that current diagonal element pivot
  else,
    keep that as pivot
- After finding pivot, divide that entire row with that pivot value to get '1' at pivot place.
- Traverse in the rows below the current pivot row and make them '0'.
By,
transforming each row by Subtracting each row with [value (i.e value below pivot) * current pivot row]

In [57]:
def row_echelon_form(A,B):

  det_A = np.linalg.det(A)
  if np.isclose(det_A,0) == True:
    return 'Singular Matrix'

  A = A.copy().astype('float64')
  B = B.copy().astype('float64')

  num_rows = len(A)

  M = augmented_matrix(A,B)

  for row in range(num_rows):
    pivot_candidate = M[row,row]
    if np.isclose(pivot_candidate,0) == True:
      index = get_index_of_first_non_zero_element_from_column(M,row,row)
      M = swap_rows(M,row,index)
      pivot = M[row,row]
    else:
      pivot = pivot_candidate

    M[row]=M[row]/pivot

    for j in range(row+1,num_rows):
      value = M[j,row]
      M[j] = M[j]-value*M[row]

  return M


In [58]:
A = np.array([
    [3,6,6],
    [5,3,6],
    [4,-5,8]
])
B = np.array([
    [1],
    [-10],
    [8]
])

row_echelon_form(A,B)

array([[ 1.        ,  2.        ,  2.        ,  0.33333333],
       [-0.        ,  1.        ,  0.57142857,  1.66666667],
       [ 0.        ,  0.        ,  1.        ,  3.81410256]])

<a name='6'></a>
## 5 - Back Substitution

After attaining row echelon form, solve for variable starting from last row and progressing upwards. Goal of Back Substitution is to make all the values to zeros above the pivot elements and resulting in a diagonal matrix with constant values as solution.

First we start with last row. Using last row we try to reduce top rows to get values as zeros.

For example if pivot is (2,2) then through back propagation (1,2) becomes '0' and then (0,2) becomes '0'.

This can be achieved through,
for each row above the current row, transform that above rows by subtracting them with value above pivot with the [value (i.e value above pivot) * current pivot row]

In [59]:
def back_substitution(M):

  M = M.copy()
  num_rows = M.shape[0]

  for row in reversed(range(num_rows)):
    current_pivot_row = M[row]
    index = get_index_of_first_non_zero_element_from_row(M,row,augmented=True)
    for j in range(row):
      row_to_reduce = M[j]
      value = row_to_reduce[index]
      #value = M[index,j]
      row_to_reduce = row_to_reduce - value * current_pivot_row
      M[j,:] = row_to_reduce

  solution = M[:,-1]
  return solution

<a name='7'></a>
## Function for Compiling Gaussian Elimination Algorithm

This integrates all the steps. Solve a linear system represented by an augmented matrix using Gaussian elimination method.

Comprises of Row echelon function and Back Substitution

In [60]:
def gaussian_elimination(A, B):
    row_echelon_M = row_echelon_form(A, B)
    if isinstance(row_echelon_M, str):
        return row_echelon_M  # e.g., 'Singular Matrix'

    return back_substitution(row_echelon_M)


## Testing with System of Equations

In [61]:
A = np.array([[1, 1],
              [4, 2]])

B = np.array([[4],
              [5]])

sol = gaussian_elimination(A, B)
print("Solution:", sol)
print(np.linalg.solve(A,B))

Solution: [-1.5  5.5]
[[-1.5]
 [ 5.5]]


In [63]:
A = np.array([
    [3,6,6],
    [5,3,6],
    [4,-5,8]
])
B = np.array([
    [1],
    [-10],
    [8]
])

print(np.linalg.solve(A,B))
print(gaussian_elimination(A,B))

[[-6.26923077]
 [-0.51282051]
 [ 3.81410256]]
[-6.26923077 -0.51282051  3.81410256]
